---
title: "00-Use Stata in Quarto(ipynb)"
author: "Simon Zhou"
date: "2025-04-29"
format: 
    html:
        code-fold: true
        fig_caption: true
        number-sections: true
        toc: true
        toc-depth: 2
---

# Python与Stata联合使用

## 前言

在Stata官网冲浪的时候，看到他们推出了Python与Stata联合使用的功能，于是就想试试。

Stata是一个强大的统计分析软件，广泛应用于社会科学、经济学、医学等领域。它提供了丰富的统计分析功能和数据处理工具，适合进行复杂的数据分析和建模工作。

在使用Stata的时候，我们可能会需要在Python中调用Stata的功能。PyStata就是一个可以让我们在Python中使用Stata的工具。它允许我们在Python中运行Stata命令，并且可以将结果返回到Python中进行进一步的分析和处理。

## 关于PyStata

[Stata 官网关于 pystata 的介绍](https://www.stata.com/features/overview/pystata-python-integration/)

PyStata 是 Stata17 中引入的一个新概念，它涵盖了所有 Stata 和 Python 的交互方式。

事实上从 Stata16 开始，我们就可以在 Stata 中调用 Python 代码，并通过 Stata 函数接口（ sfi 模块）实现 Python 与 Stata 核心功能的交互；但 Stata17 通过允许我们通过导入一个新的 Python 包（pystata）从一个独立的 Python 环境中调用 Stata ，这大大扩展二者的交互功能，使我们可以在基于或支持 IPython 内核的环境中（例如：Jupyter Notebook 、Jupyter Lab 、Spyder 、PyCharm 、VScode 等）更加方便地调用 Stata 和 Mata。

### pystata与stata_kernel的区别

自 Stata17 之后，官方推出了一种 StataS与 Python 的全新交互方式，而 stata_kernel 是一个第三方项目。

即通过在 Python 环境中直接安装 pystata 模块，便能在 Python 环境中直接调用 Stata17 的命令。而 stata_kernel 是通过在 Jupyter Notebook 中安装 stata_kernel 模块，来实现 Python 与 Stata 的交互。

在使用Quarto制作本网站时，可以编译 `.qmd` 和 `.ipynb` 文件，生成 `.html` 和 `.pdf` 文件，就想是否可以 Quarto 中利用 `.ipynb` 文件来调用 Stata 然后编译成 `.html` 文件再在网站中展示出来。

于是有了这篇笔记。

步骤主要分为四部分：

1. 将 Stata 添加到系统环境变量中
2. 在 Python 中安装 PyStata
3. 在 Jupyter Notebook 中使用PyStata
4. 在 Quarto 中使用 PyStata

## 前置要求

- 请先安装好 Stata 17 或更高版本，且最好是无限制的版本，因为有很多网上的资源是破解的，可能会有一些限制。
- Stata17/18/19 软件必须具备有效的许可证，否则无法调用
- 拥有基于或支持 IPython 内核的 Python 环境（建议使用 Jupyter Lab 或 VScode）
- Python 3.7 或更高版本（建议使用 Anaconda 进行安装和管理）

## 依赖项

要使用 pystata 包的完整功能，需要安装以下 Python 包：

- NumPy 1.9 或更高版本
- pandas 0.15 或更高版本
    - 如果您仅计划通过调用 `stata` 模块中的 `run()` 方法执行 Stata 命令，则无需安装 NumPy 和 pandas 包。
    - 然而，如果需要调用 `stata` 模块中用于在 Stata 和 Python 之间传递数据和结果的方法，则必须安装这些包。
- IPython 5.0 或更高版本
    - 如果您想使用魔法命令，则需要安装 IPython 包。

## Stata 添加到系统环境变量中

### 方法一：手动添加
1. 找到 Stata 的安装目录，通常在 `C:\Program Files\Stata18` 或 `C:\Program Files (x86)\Stata18`。
2. 复制该目录的路径。
3. 右键点击“此电脑”或“计算机”，选择“属性”。
4. 点击“高级系统设置”。
5. 在“系统属性”窗口中，点击“环境变量”。
6. 在“系统变量”部分，找到名为“Path”的变量，选中它并点击“编辑”。
7. 在“编辑环境变量”窗口中，点击“新建”，然后粘贴 Stata 的安装目录路径。
8. 点击“确定”保存更改，关闭所有窗口。
9. 重新启动命令提示符或 PowerShell，以使更改生效。
10. 在命令提示符中输入 `stata`，如果 Stata 启动，则说明添加成功。

### 方法二：使用命令行添加
1. 打开命令提示符或 PowerShell。
2. 输入以下命令，将 `C:\Program Files\Stata18` 替换为 Stata 的安装目录：

```bash
setx PATH "%PATH%;C:\Program Files\Stata17"
```

3. 按下回车键执行命令。
4. 关闭命令提示符或 PowerShell。
5. 重新打开命令提示符或 PowerShell，以使更改生效。
6. 在命令提示符中输入 `stata`，如果 Stata 启动，则说明添加成功。

### 另一种办法

这个办法也有一定的普及程度，但是可能不太好用，对于新手来说会有些难以理解。可以参考连玉君的[珠联璧合：Jupyter Notebook 和 Stata 之融合](https://zhuanlan.zhihu.com/p/93100740)。

主要步骤如下：

1. 找到 Stata 的安装目录，通常在 `C:\Program Files\Stata18` 或 `C:\Program Files (x86)\Stata18`。
2. 在该目录下找到 `StataMP.exe` 或 `StataSE.exe` 文件。
3. 将该文件的路径复制下来。
4. 以管理员身份打开电脑的 `Windows Powershell` 。
5. 在 `PS C:\WINDOWS\system32>` 后输入以下命令（将路径转到 Stata 安装目录下）：

在 Windows PowerShell 执行 cd 命令，以进入 stata 程序安装的路径。cd 命令后接上步所获取的 stata 安装路径。根据个人电脑安装路径不同有所差异。路径请以英文引号包围，这样可以避免路径文件夹名称中包含空格导致无法顺利进入目标路径。

```powershell
cd "C:\Program Files\Stata18"
```

实际效果应该如下：

```powershell
PS C:\WINDOWS\system32> cd 'C:\Program Files\Stata18'
PS C:\Program Files\Stata18>
```

6. 输入以下命令来将 stata 添加到命令行注册表中：

```powershell
.\StataMP-64.exe /Register
```

*需要注意的是： `.\StataMP-64.exe /Register` 中的 `.\StataMP-64.exe` 部分，根据个人电脑安装 Stata17+ 版本有所差异。我电脑安装的是 MP 版，所以为 `.\StataMP-64.exe`。如果安装的是 SE 版，应该为 `.\StataSE-64.exe`。*

这里实测效果不太好，不知道为什么，注册成功但是命令行输入 `stata` 还是无法打开 Stata。

# 安装PyStata

## 方法一：使用 pip 配置 pystata

PyStata 可以通过 pip 安装。Windows 可以使用以下命令进行安装：

```bash
pip install --upgrade --user stata_setup
```

macOS 或 Unix 系统可以使用以下命令进行安装：

```bash
$ pip install --upgrade --user stata_setup
```

### 配置 Stata

假设你的 Stata 安装在 STATA_SYSDIR 目录下，并且使用的是 Stata/MP 版本。你可以在 Python 环境中按如下方式配置 Stata：

如果 Stata 配置正确，`stata_setup.config()` 将返回如下的启动画面，其中包含 Stata 的徽标和初始化消息。

In [1]:
import stata_setup
stata_setup.config('C:/Program Files/Stata18', 'mp')


  ___  ____  ____  ____  ____ ®
 /__    /   ____/   /   ____/      18.0
___/   /   /___/   /   /___/       MP—Parallel Edition

 Statistics and Data Science       Copyright 1985-2023 StataCorp LLC
                                   StataCorp
                                   4905 Lakeway Drive
                                   College Station, Texas 77845 USA
                                   800-STATA-PC        https://www.stata.com
                                   979-696-4600        stata@stata.com

Stata license: Unlimited-user 2-core network, expiring  8 Apr 2026
Serial number: 501809376090
  Licensed to: ausa
               NJU

Notes:
      1. Unicode is supported; see help unicode_advice.
      2. More than 2 billion observations are allowed; see help obs_advice.
      3. Maximum number of variables is set to 5,000 but can be increased;
          see help set_maxvar.


如果你不想看到初始化的信息，可以使用如下命令将其隐藏：

```python
stata_setup.config('YOUR_STATA_SYSDIR', 'mp', splash=False)
```

## 方法二：将 pystata 添加到 sys.path

找到 pystata 包的最直接方法是将 pystata 子目录的位置添加到 Python 的模块搜索路径中。在你的 Python 环境中，你可以输入

```python
import sys
sys.path.append('STATA_SYSDIR/utilities')
from pystata import config
config.init('mp')
```

如果配置正确，`config.init()` 应该返回无错误，并显示同上面一样的的启动画面，其中包含 Stata 的徽标和初始化消息。如果您想隐藏这些消息，可以将 splash 参数设置为 False。

更多的安装和配置信息可以访问：[pystata](https://www.stata.com/python/pystata19/install.html)

# 使用PyStata

在安装完成后，我们可以在Python中导入PyStata，并使用它来运行Stata命令。以下是一个简单的示例：

首先，我们在 Stata 中加载数据集、描述其内容并显示其时间序列设置。

In [3]:
%%stata
use https://www.stata-press.com/data/r19/lutkepohl2
describe
tsset


. use https://www.stata-press.com/data/r19/lutkepohl2
(Quarterly SA West German macro data, Bil DM, from Lutkepohl 1993 Table E.1)

. describe

Contains data from https://www.stata-press.com/data/r19/lutkepohl2.dta
 Observations:            92                  Quarterly SA West German macro
                                                data, Bil DM, from Lutkepohl
                                                1993 Table E.1
    Variables:            10                  4 Dec 2024 14:31
-------------------------------------------------------------------------------
Variable      Storage   Display    Value
    name         type    format    label      Variable label
-------------------------------------------------------------------------------
inv             int     %8.0g                 Investment
inc             int     %8.0g                 Income
consump         int     %8.0g                 Consumption
qtr             byte    %tq                   Quarter
ln_inv          floa

In [4]:
import pystata
from pystata import stata